In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style1/128/b0/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 5000
start_epoch = 0
train_batch = 32
test_batch = 300
lr = 0.01
schedule = [1500, 3000]
momentum = 0
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b0/to_pggan/l2sp_style1' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
alpha = 0.1
beta = 0.1
fc_name = 'fc.'

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot_style1')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
source_model = EfficientNet.from_name(model_name, num_classes=num_classes, override_params={'dropout_rate':0.0})
model = EfficientNet.from_name(model_name, num_classes=num_classes, override_params={'dropout_rate':0.0})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    source_model.load_state_dict(torch.load(pretrained)['state_dict'])
    model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b0/checkpoint.pth.tar'


In [9]:
source_model.to('cuda')
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.01M


In [10]:
for param in source_model.parameters():
    param.requires_grad = False
source_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [11]:
source_model_weights = {}
for name, param in source_model.named_parameters():
    source_model_weights[name] = param.detach()

# Model tuning

In [12]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - source_model_weights[name]) ** 2
    return sp_loss

In [13]:
summary(model, input_size=(3,128,128),device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 129, 129]               0
Conv2dStaticSamePadding-2           [-1, 32, 64, 64]             864
         GroupNorm-3           [-1, 32, 64, 64]              64
MemoryEfficientSwish-4           [-1, 32, 64, 64]               0
         ZeroPad2d-5           [-1, 32, 66, 66]               0
Conv2dStaticSamePadding-6           [-1, 32, 64, 64]             288
         GroupNorm-7           [-1, 32, 64, 64]              64
MemoryEfficientSwish-8           [-1, 32, 64, 64]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 32, 1, 1]             288
         I

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [16]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss = loss_main + alpha*loss_sp + beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss = loss_main + alpha*loss_sp + beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [19]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        test_loss, test_acc = test(val_target_loader, model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, model, criterion, epoch, use_cuda)

        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        

        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 5000] LR: 0.010000
1/13 | Total:0:00:03 | ETA:0:00:39 | Loss:1.8725745677947998 | top1:78.125
11/13 | Total:0:00:07 | ETA:0:00:02 | Loss:1.2964087995615872 | top1:67.04545593261719
13/13 | Total:0:00:07 | ETA:0:00:01 | Loss:1.2528382142384846 | top1:66.40625


/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


107/107 | Total:0:00:48 | ETA:0:00:00 | Loss:1.093954790975446 | top1:37.112152099609375
26/26 | Total:0:00:32 | ETA:0:00:00 | Loss:0.2762604086444928 | top1:87.06410217285156

Epoch: [2 | 5000] LR: 0.017000
1/13 | Total:0:00:01 | ETA:0:00:20 | Loss:0.8805246353149414 | top1:59.375
11/13 | Total:0:00:06 | ETA:0:00:02 | Loss:0.6339508891105652 | top1:65.34091186523438
13/13 | Total:0:00:06 | ETA:0:00:01 | Loss:0.6310860961675644 | top1:65.88542175292969

Epoch: [3 | 5000] LR: 0.024000
1/13 | Total:0:00:02 | ETA:0:00:25 | Loss:0.44686830043792725 | top1:75.0
11/13 | Total:0:00:07 | ETA:0:00:02 | Loss:0.5058955739844929 | top1:71.0227279663086
13/13 | Total:0:00:08 | ETA:0:00:01 | Loss:0.507829430202643 | top1:70.83333587646484

Epoch: [4 | 5000] LR: 0.031000
1/13 | Total:0:00:01 | ETA:0:00:18 | Loss:0.5236808061599731 | top1:75.0
11/13 | Total:0:00:06 | ETA:0:00:02 | Loss:0.5071914819153872 | top1:73.29545593261719
13/13 | Total:0:00:06 | ETA:0:00:01 | Loss:0.4993914837638537 | top1:73.4

1/13 | Total:0:00:01 | ETA:0:00:24 | Loss:0.5037416219711304 | top1:81.25
11/13 | Total:0:00:07 | ETA:0:00:02 | Loss:0.45574970136989246 | top1:85.51136779785156
13/13 | Total:0:00:08 | ETA:0:00:01 | Loss:0.45658762753009796 | top1:85.15625

Epoch: [32 | 5000] LR: 0.079997
1/13 | Total:0:00:02 | ETA:0:00:25 | Loss:0.3754367232322693 | top1:90.625
11/13 | Total:0:00:07 | ETA:0:00:02 | Loss:0.44764461571520026 | top1:88.06818389892578
13/13 | Total:0:00:08 | ETA:0:00:01 | Loss:0.4584944223364194 | top1:86.97917175292969

Epoch: [33 | 5000] LR: 0.079997
1/13 | Total:0:00:02 | ETA:0:00:36 | Loss:0.42526280879974365 | top1:90.625
11/13 | Total:0:00:08 | ETA:0:00:02 | Loss:256.6073542535305 | top1:86.6477279663086
13/13 | Total:0:00:09 | ETA:0:00:01 | Loss:279.99515692641336 | top1:86.45833587646484

Epoch: [34 | 5000] LR: 0.079996
1/13 | Total:0:00:01 | ETA:0:00:18 | Loss:528.7898559570312 | top1:84.375
11/13 | Total:0:00:06 | ETA:0:00:02 | Loss:488.6182889071378 | top1:85.79545593261719
13

11/13 | Total:0:00:08 | ETA:0:00:02 | Loss:3.819406336004084 | top1:85.2272720336914
13/13 | Total:0:00:09 | ETA:0:00:01 | Loss:3.7897194822629294 | top1:85.41667175292969

Epoch: [61 | 5000] LR: 0.079981
1/13 | Total:0:00:02 | ETA:0:00:26 | Loss:3.463434934616089 | top1:87.5
11/13 | Total:0:00:08 | ETA:0:00:02 | Loss:3.172880866310813 | top1:88.3522720336914
13/13 | Total:0:00:09 | ETA:0:00:01 | Loss:3.175708075364431 | top1:87.23958587646484

Epoch: [62 | 5000] LR: 0.079980
1/13 | Total:0:00:02 | ETA:0:00:31 | Loss:2.8963522911071777 | top1:87.5
11/13 | Total:0:00:06 | ETA:0:00:02 | Loss:2.788075663826682 | top1:82.38636779785156
13/13 | Total:0:00:07 | ETA:0:00:01 | Loss:2.7609231074651084 | top1:83.33333587646484

Epoch: [63 | 5000] LR: 0.079979
1/13 | Total:0:00:02 | ETA:0:00:27 | Loss:2.496934413909912 | top1:84.375
11/13 | Total:0:00:07 | ETA:0:00:02 | Loss:2.4345631816170434 | top1:83.5227279663086
13/13 | Total:0:00:08 | ETA:0:00:01 | Loss:2.4105544288953147 | top1:83.59375

E

...

1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.15830209851264954 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.14639474248344247 | top1:99.71591186523438
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.14531441467503706 | top1:99.73958587646484

Epoch: [4865 | 5000] LR: 0.000002
1/13 | Total:0:00:01 | ETA:0:00:14 | Loss:0.25870761275291443 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.16758919845927844 | top1:98.86363983154297
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.16523837794860205 | top1:98.95833587646484

Epoch: [4866 | 5000] LR: 0.000002
1/13 | Total:0:00:01 | ETA:0:00:14 | Loss:0.17384284734725952 | top1:96.875
11/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.15386248989538712 | top1:98.01136779785156
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.16529934853315353 | top1:97.65625

Epoch: [4867 | 5000] LR: 0.000002
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.22415336966514587 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.1734540658918294 | top1:98.2

11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.19811617853966626 | top1:97.44318389892578
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.19318194004396597 | top1:97.65625

Epoch: [4894 | 5000] LR: 0.000001
1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.16423621773719788 | top1:96.875
11/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.18762324479493228 | top1:97.7272720336914
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.18499322359760603 | top1:97.91667175292969

Epoch: [4895 | 5000] LR: 0.000001
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.18605276942253113 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.16664819554849106 | top1:98.29545593261719
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.17240943759679794 | top1:97.91667175292969

Epoch: [4896 | 5000] LR: 0.000001
1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.12427280843257904 | top1:100.0
11/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.16873747245831924 | top1:97.7272720336914
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.16599144786596298 

1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.16082167625427246 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.15642910924824802 | top1:98.86363983154297
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.15373321995139122 | top1:98.95833587646484

Epoch: [4923 | 5000] LR: 0.000001
1/13 | Total:0:00:01 | ETA:0:00:13 | Loss:0.15697385370731354 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.17249156737869437 | top1:97.15909576416016
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.16981030690173307 | top1:97.39583587646484

Epoch: [4924 | 5000] LR: 0.000001
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.2645891010761261 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.18887661397457123 | top1:97.7272720336914
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.1897652397553126 | top1:97.39583587646484

Epoch: [4925 | 5000] LR: 0.000001
1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.1735459268093109 | top1:96.875
11/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.18569860742850738 | to

11/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.16735187579285016 | top1:98.29545593261719
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.1637506646414598 | top1:98.4375
107/107 | Total:0:00:20 | ETA:0:00:00 | Loss:0.5022017209329338 | top1:86.8380126953125
26/26 | Total:0:00:14 | ETA:0:00:00 | Loss:0.14805562478991655 | top1:98.9358901977539

Epoch: [4952 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:08 | Loss:0.2324487715959549 | top1:93.75
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.17854788222096182 | top1:97.44318389892578
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.1762862652540207 | top1:97.65625

Epoch: [4953 | 5000] LR: 0.000000
1/13 | Total:0:00:01 | ETA:0:00:13 | Loss:0.3082899749279022 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.2131277864629572 | top1:96.30682373046875
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.2101875183482965 | top1:96.61458587646484

Epoch: [4954 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.15697839856147766 | to

1/13 | Total:0:00:01 | ETA:0:00:14 | Loss:0.1487473100423813 | top1:100.0
11/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.20296580547636206 | top1:96.875
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.19656285643577576 | top1:97.13542175292969

Epoch: [4981 | 5000] LR: 0.000000
1/13 | Total:0:00:01 | ETA:0:00:13 | Loss:0.1387856900691986 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.18845537846738641 | top1:97.44318389892578
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.19625377903381983 | top1:97.39583587646484

Epoch: [4982 | 5000] LR: 0.000000
1/13 | Total:0:00:01 | ETA:0:00:15 | Loss:0.13680851459503174 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.17469000003554605 | top1:97.7272720336914
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.18420006583134332 | top1:97.13542175292969

Epoch: [4983 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.13060078024864197 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.1767017516222867 | top1:97.727272033